In [19]:
import pandas as pd
import operator
import numpy as np
import copy
import random as rd
rd.seed(10)

In [4]:
def helper_load_train(filename):
    user_dict_list = {}
    item_dict = set()
    item_dict_list = {}
    trainUser, trainItem = [], []

    with open(filename) as f:
        for line in f.readlines():
            line = line.strip('\n').split(' ')
            if len(line) == 0:
                continue
            line = [int(i) for i in line]
            user = line[0]
            items = line[1:]
            item_dict.update(items)
            # LGN
            trainUser.extend([user] * len(items))
            trainItem.extend(items)
            if len(items) == 0:
                continue
            user_dict_list[user] = items

            for item in items:
                if item in item_dict_list.keys():
                    item_dict_list[item].append(user)
                else:
                    item_dict_list[item] = [user]

    return user_dict_list, item_dict, item_dict_list, trainUser, trainItem

In [5]:
def convert(item_dict):
    user_list = []
    item_list = []
    for key, value in item_dict.items():
        for user in value:
            user_list.append(user)
            item_list.append(key)
            
    return user_list, item_list

In [6]:
train_user_list, _, train_item_list, _, _ = helper_load_train("train.txt")
valid_user_list, _, valid_item_list, _, _ = helper_load_train("valid.txt")
test_ood_user_list, _, test_ood_item_list, _, _ = helper_load_train("test_ood.txt")
test_id_user_list, _, test_id_item_list, _ , _ = helper_load_train("test_id.txt")

train_user, train_item = convert(train_item_list)
valid_user, valid_item = convert(valid_item_list)
test_id_user, test_id_item = convert(test_id_item_list)
test_ood_user, test_ood_item = convert(test_ood_item_list)

user_list = train_user + valid_user + test_id_user + test_ood_user
item_list = train_item + valid_item + test_id_item + test_ood_item

In [7]:
df = pd.DataFrame(data = {'user': user_list, 'item': item_list})

In [8]:
total_item_dict = {}
for index, row in df.iterrows():
    item = row['item']
    user = row['user']
    if item in total_item_dict.keys():
        total_item_dict[item].append(user)
    else:
        total_item_dict[item] = [user]

In [9]:
pop_item = {key: len(value) for key, value in total_item_dict.items()}

In [14]:
sorted_pop_item = dict(sorted(pop_item.items(), key=operator.itemgetter(1),reverse=True))
sorted_items = np.array(list(sorted_pop_item.keys()))
each_chunk = int(len(sorted_items)/50)

temp = list(np.linspace(each_chunk,each_chunk*49,49))
temp = [int(x) for x in temp]
chunks = np.split(sorted_items,temp)

In [15]:
def sample_data(chunks = chunks, gamma = 1, n_max = 12, total_item_dict = total_item_dict):

    test_dict = {}
    left_dict = copy.deepcopy(total_item_dict)

    for i in range(50):
        chunk = chunks[i]
        
        for item in chunk:
            
            expected_sample = n_max * gamma ** (-(i-1)/(50-1))
            total = len(total_item_dict[item])
            sample_weight = expected_sample / total
            
            for user in left_dict[item]:
                
                rad=rd.random()
                # if not sampled, pass
                if rad > sample_weight:
                    continue
                # if this is the only one left interaction, pass
                if total == 1:
                    continue
                # else add to the dictionary
                if item in test_dict.keys():
                    test_dict[item].append(user)
                else:
                    test_dict[item] = [user]
                
                
                left_dict[item].remove(user)
                    
    return test_dict, left_dict

In [20]:
test_dict_1, total_dict_after_1 = \
sample_data(chunks = chunks, gamma = 1, n_max = 12, total_item_dict = total_item_dict)